In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pydicom
import os
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from datetime import datetime
import os
import pandas as pd
from sklearn.model_selection import train_test_split



In [33]:
# import pre_processing 
from pre_processing import HandScanDataset2, transform, validation_transform, train_df, valid_df

In [34]:
class ConvBlock3D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock3D, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

In [35]:
class Encoder3D(nn.Module):
    def __init__(self, in_channels):
        super(Encoder3D, self).__init__()
        self.encoder = nn.Sequential(
            ConvBlock3D(in_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.MaxPool3d(kernel_size=2, stride=2),
            ConvBlock3D(32, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool3d(kernel_size=2, stride=2),
            ConvBlock3D(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool3d(kernel_size=2, stride=2),
            ConvBlock3D(128, 256, kernel_size=3, stride=1, padding=1),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.encoder(x)

In [36]:
class Decoder3D(nn.Module):
    def __init__(self, in_channels):
        super(Decoder3D, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(in_channels, 128, kernel_size=2, stride=2),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(64, 32, kernel_size=2, stride=2),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(32, 1, kernel_size=2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)

In [37]:
class MaskedAutoencoder3D(nn.Module):
    def __init__(self, in_channels):
        super(MaskedAutoencoder3D, self).__init__()
        self.encoder = Encoder3D(in_channels)
        self.decoder = Decoder3D(256)

    def forward(self, x, mask):
        x_masked = x * mask
        encoded = self.encoder(x_masked)
        decoded = self.decoder(encoded)
        return decoded

In [38]:
def create_mask(shape, mask_ratio=0.7):
    mask = np.ones(shape)
    num_elements = shape[1] * shape[2] * shape[3]
    num_masked = int(mask_ratio * num_elements)
    indices = np.random.choice(num_elements, num_masked, replace=False)
    mask.reshape(-1)[indices] = 0
    return torch.tensor(mask, dtype=torch.float32)

In [43]:
class EarlyStopper:
    def __init__(self, patience=3, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_val_loss = float('inf')
        self.early_stop = False
        self.activation_count = 0

    def __call__(self, val_loss):
        if val_loss < self.best_val_loss - self.min_delta:
            self.best_val_loss = val_loss
            self.counter = 0
            self.activation_count += 1
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        return self.early_stop

def train_step(model, dataloader, loss_fn, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        mask = create_mask(images.shape)
        outputs = model(images, mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(dataloader)
    train_acc = correct / total
    return train_loss, train_acc

def val_step(model, dataloader, loss_fn):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            mask = create_mask(images.shape)
            outputs = model(images, mask)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss /= len(dataloader)
    val_acc = correct / total
    return val_loss, val_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          val_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          results_path: str = "./results"):

    results = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    early_stopper = EarlyStopper(patience=3, min_delta=0.03)

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer)
        val_loss, val_acc = val_step(model, val_dataloader, loss_fn)

        if early_stopper(val_loss):
            if early_stopper.activation_count == 1:
                print(f"\nEarly stopping criteria reached!\nSaving model parameters to {results_path}/{timestamp}_optimal_model_weights_epoch_{epoch + 1}.pth\n")
                opt_MLP_filepath = f"{results_path}/{timestamp}_optimal_model_weights_epoch_{epoch + 1}.pth"
                Early_Stop_Epoch = epoch + 1
                torch.save(model.state_dict(), opt_MLP_filepath)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"val_loss: {val_loss:.4f} | "
            f"val_acc: {val_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)

    return results, Early_Stop_Epoch

In [44]:
 if __name__ == '__main__':
    # Assuming 'HandScanDataset2' and 'MaskedAutoencoder3D' are defined elsewhere
    training_data_dir = "/Users/eleanorbolton/Library/CloudStorage/OneDrive-UniversityofLeeds/t1_vibe_we_hand_subset/"
    csv_path = os.path.join(training_data_dir, 'training_labels_subset.csv')
    labels_df = pd.read_csv(csv_path)
    train_df, valid_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['progression'])
    train_df.to_csv(os.path.join(training_data_dir, 'train_split.csv'), index=False)
    valid_df.to_csv(os.path.join(training_data_dir, 'valid_split.csv'), index=False)
    

    train_dataset = HandScanDataset2(labels_df=train_df, data_dir=training_data_dir, transform=transform)
    valid_dataset = HandScanDataset2(labels_df=valid_df, data_dir=training_data_dir, transform=validation_transform)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MaskedAutoencoder3D(in_channels=1).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train(model, train_loader, valid_loader, optimizer, epochs=50, results_path="./results")

{'CCP_120': 1, 'CCP_117': 0, 'CCP_73': 0, 'CCP_386': 1, 'CCP_644': 1, 'CCP_NG_166': 0, 'CCP_NG_42': 1, 'CCP_138': 0, 'CCP_874': 0, 'CCP_907': 1, 'CCP_NG_137': 1, 'CCP_647': 1, 'CCP_262': 0, 'CCP_541': 0, 'CCP_89': 0, 'CCP_202': 0, 'CCP_NG_56': 0, 'CCP_821': 1, 'CCP_66': 0, 'CCP_471': 0, 'CCP_34': 0, 'CCP_181': 1, 'CCP_557': 1, 'CCP_416': 0, 'CCP_NG_207': 0, 'CCP_568': 0, 'CCP_753': 0, 'CCP_NG_181': 0, 'CCP_81': 0, 'CCP_415': 1, 'CCP_NG_8': 1, 'CCP_283': 1, 'CCP_906': 0, 'CCP_968': 0, 'CCP_664': 0, 'CCP_736': 0, 'CCP_355': 0, 'CCP_NG_104': 1, 'CCP_247': 1, 'CCP_NG_188': 1, 'CCP_976': 0, 'CCP_NG_214': 0, 'CCP_824': 1, 'CCP_62': 0, 'CCP_NG_36': 0, 'CCP_802': 0, 'CCP_NG_172': 0, 'CCP_873': 0, 'CCP_207': 0, 'CCP_167': 1, 'CCP_944': 1, 'CCP_133': 1, 'CCP_NG_60': 0, 'CCP_1000': 0, 'CCP_252': 1, 'CCP_672': 0, 'CCP_531': 0, 'CCP_NG_175': 1, 'CCP_NG_107': 1, 'CCP_53': 0, 'CCP_NG_106': 1, 'CCP_105': 0, 'CCP_507': 1, 'CCP_405': 0, 'CCP_172': 1, 'CCP_901': 1, 'CCP_212': 1, 'CCP_485': 0, 'CCP_185': 

  0%|          | 0/50 [00:00<?, ?it/s]

Best instance number: 57
Patient ID: CCP_105, Image shape: (96, 384, 512)
Best instance number: 109
Patient ID: CCP_780, Image shape: (96, 384, 512)
Best instance number: 67
Patient ID: CCP_531, Image shape: (96, 384, 512)
Best instance number: 124
Patient ID: CCP_672, Image shape: (96, 384, 512)


RuntimeError: Expected target size [4, 96, 96, 96], got [4]

In [22]:
if __name__ == '__main__':
    # Reading the CSV file
    training_data_dir = "/Users/eleanorbolton/Library/CloudStorage/OneDrive-UniversityofLeeds/t1_vibe_we_hand_subset/" 
    csv_path = os.path.join(training_data_dir, 'training_labels_subset.csv')
    labels_df = pd.read_csv(csv_path)

    # Split the data into training and validation sets
    train_df, valid_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['progression'])

    # Save the splits for reference
    train_df.to_csv(os.path.join(training_data_dir, 'train_split.csv'), index=False)
    valid_df.to_csv(os.path.join(training_data_dir, 'valid_split.csv'), index=False)
    train_dataset = HandScanDataset2(labels_df=train_df, data_dir=training_data_dir, transform=transform)
    valid_dataset = HandScanDataset2(labels_df=valid_df, data_dir=training_data_dir, transform=validation_transform)

    # Creating data loaders
    batch_size = 4
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    
    model = MaskedAutoencoder3D(in_channels=1)
    train_mae(model, train_loader, valid_loader, num_epochs=50, learning_rate=0.001)

{'CCP_120': 1, 'CCP_117': 0, 'CCP_73': 0, 'CCP_386': 1, 'CCP_644': 1, 'CCP_NG_166': 0, 'CCP_NG_42': 1, 'CCP_138': 0, 'CCP_874': 0, 'CCP_907': 1, 'CCP_NG_137': 1, 'CCP_647': 1, 'CCP_262': 0, 'CCP_541': 0, 'CCP_89': 0, 'CCP_202': 0, 'CCP_NG_56': 0, 'CCP_821': 1, 'CCP_66': 0, 'CCP_471': 0, 'CCP_34': 0, 'CCP_181': 1, 'CCP_557': 1, 'CCP_416': 0, 'CCP_NG_207': 0, 'CCP_568': 0, 'CCP_753': 0, 'CCP_NG_181': 0, 'CCP_81': 0, 'CCP_415': 1, 'CCP_NG_8': 1, 'CCP_283': 1, 'CCP_906': 0, 'CCP_968': 0, 'CCP_664': 0, 'CCP_736': 0, 'CCP_355': 0, 'CCP_NG_104': 1, 'CCP_247': 1, 'CCP_NG_188': 1, 'CCP_976': 0, 'CCP_NG_214': 0, 'CCP_824': 1, 'CCP_62': 0, 'CCP_NG_36': 0, 'CCP_802': 0, 'CCP_NG_172': 0, 'CCP_873': 0, 'CCP_207': 0, 'CCP_167': 1, 'CCP_944': 1, 'CCP_133': 1, 'CCP_NG_60': 0, 'CCP_1000': 0, 'CCP_252': 1, 'CCP_672': 0, 'CCP_531': 0, 'CCP_NG_175': 1, 'CCP_NG_107': 1, 'CCP_53': 0, 'CCP_NG_106': 1, 'CCP_105': 0, 'CCP_507': 1, 'CCP_405': 0, 'CCP_172': 1, 'CCP_901': 1, 'CCP_212': 1, 'CCP_485': 0, 'CCP_185': 

AttributeError: 'CrossEntropyLoss' object has no attribute 'backward'